[View in Colaboratory](https://colab.research.google.com/github/krisbuote/philosopher-king-rnn/blob/master/philosopher_king_RNN.ipynb)

# Philosopher King Karpathy
Minimal character-level language model with a Vanilla Recurrent Neural Network, in Python/numpy.

Code: https://github.com/karpathy/char-rnn

Tutorial: http://karpathy.github.io/2015/05/21/rnn-effectiveness/


In [3]:
# Upload dataset and model params if applicable
# Files must be located in /downloads
from google.colab import files
uploaded = files.upload()


for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving n_p_hprev_smoothloss.p to n_p_hprev_smoothloss.p
Saving philosopher_params.p to philosopher_params.p
Saving philosophy_data_new.txt to philosophy_data_new.txt
User uploaded file "n_p_hprev_smoothloss.p" with length 2778 bytes
User uploaded file "philosopher_params.p" with length 864233 bytes
User uploaded file "philosophy_data_new.txt" with length 12390598 bytes


In [0]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License

Adapted by Kris Buote @krisbuote
"""
import numpy as np
import cPickle as pickle
  
resume = True # Set to True if loading saved model parameters
saving = True # Set to True if you want to save model parameters

# data I/O
data = open('philosophy_data_new.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 30 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
if resume == True: #if loading saved parameters
  params = pickle.load(open('philosopher_params.p','rb'))
  Wxh = params[0]
  Whh = params[1]
  Why = params[2]
  bh = params[3]
  by = params[4]
  
else:
  Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
  Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
  Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
  bh = np.zeros((hidden_size, 1)) # hidden bias
  by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in xrange(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in xrange(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

if resume == True: #Load iteration, data pointer, hprev, smooth loss from last run
  n_p_hprev_smoothloss = pickle.load(open('n_p_hprev_smoothloss.p','rb'))
  n = n_p_hprev_smoothloss[0]
  p = n_p_hprev_smoothloss[1]
  hprev = n_p_hprev_smoothloss[2]
  smooth_loss = n_p_hprev_smoothloss[3]
  
else:
  n, p = 0, 0
  smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

  
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad

while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]] #current character as int
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]] #target: next character as int

  # sample from the model now and then
  sample_rate = 5000 # how often (in iterations) to produce samples and save params
  
  if n % sample_rate == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print '----\n %s \n----' % (txt, )

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % sample_rate == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
    
  #Save parameters, loss counter, data pointer, iteration number
  if saving == True and n % sample_rate == 0: 
    params = [Wxh, Whh, Why, bh, by] # model parameters
    n_p_hprev_smoothloss = [n, p, hprev, smooth_loss] # iteration, data pointer, prev. hidden state
    pickle.dump(params, open('philosopher_params.p', 'wb'))
    pickle.dump(n_p_hprev_smoothloss, open('n_p_hprev_smoothloss.p', 'wb'))
    

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 12390598 characters, 132 unique.
----
 e wholdes has
may �it able, hideelmodight the beciesing exbling inthe iveap in courecous, the strevee hands? The ful his, the premald
useation, alably it lew
same caser. Helly he some we so ald of  
----
iter 270000, loss: 66.717512
----
 comupaled tharppare
 its of ald anlg of, of obs it whowe he on of of teratis act nyd wind
dehiduas, it by ector. This be wal linoth alh
in
wass all it sealionowe ine imertiend we wivith
sathike,  
----
iter 275000, loss: 64.572008
----
 my
civisciif,
will. As of to itte eses and lount buge venifs noffarfatag indity wavidive insoull lound firs. Votan cecratures lets how the warwelich bithe foucars offase as wresrantinn it in it
his 
----
iter 280000, loss: 62.644534
----
 or acampussetieveicosewiend to the is, whion oug on ak dimastating
to ariwe auributed afrnoraun reas ussertstlem, a suutuw, trater
outieniect sone a mite. xI. Ho Not all is
peag to to fuklentue tos 
----
iter 285000, loss: 63.949899
----

In [0]:
#Run this if you'd like to save the parameters to local machine's /downloads
files.download('philosopher_params.p')
files.download('n_p_hprev_smoothloss.p')

In [0]:
# gradient checking
from random import uniform
def gradCheck(inputs, target, hprev):
  global Wxh, Whh, Why, bh, by
  num_checks, delta = 10, 1e-5
  _, dWxh, dWhh, dWhy, dbh, dby, _ = lossFun(inputs, targets, hprev)
  for param,dparam,name in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby], ['Wxh', 'Whh', 'Why', 'bh', 'by']):
    s0 = dparam.shape
    s1 = param.shape
    assert s0 == s1, 'Error dims dont match: %s and %s.' % (`s0`, `s1`)
    print name
    for i in xrange(num_checks):
      ri = int(uniform(0,param.size))
      # evaluate cost at [x + delta] and [x - delta]
      old_val = param.flat[ri]
      param.flat[ri] = old_val + delta
      cg0, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)
      param.flat[ri] = old_val - delta
      cg1, _, _, _, _, _, _ = lossFun(inputs, targets, hprev)
      param.flat[ri] = old_val # reset old value for this parameter
      # fetch both numerical and analytic gradient
      grad_analytic = dparam.flat[ri]
      grad_numerical = (cg0 - cg1) / ( 2 * delta )
      rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
      print '%f, %f => %e ' % (grad_numerical, grad_analytic, rel_error)
      # rel_error should be on order of 1e-7 or less